In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

# Initialize the WebDriver
driver = webdriver.Chrome()
driver.implicitly_wait(5)
wait = WebDriverWait(driver, 10)

# Navigate to the upsrtc Bus Routes page
driver.get("https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(2)

# Function to scrape route data
def scrape_page():
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[9]/div[1]/a')))
    route_data = []  # List to store the scraped route data
    for route in routescontainer:
        route_data.append({
            'Route Name': route.text,
            'Route Link': route.get_attribute('href')
        })
    return route_data  # Return the scraped data

# Function to navigate pages
def navigate_pages():
    all_routes = []
    for page_number in range(1, 6):  # Limit to 5 pages
        try:
            routes = scrape_page()
            all_routes.extend(routes)
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class,"DC_117_pageTabs") and text()="{page_number + 1}"]')
            next_page_button.click()
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1))
            )
            time.sleep(2)  # Allow page to load
        except Exception as e:
            print(f"Pagination error at page {page_number + 1}: {e}")
            break
    return all_routes

# Function to scrape bus details
def collect_bus_details(route_data):
    alldetails = []
    for data in route_data:
        link = data['Route Link']
        route_name = data['Route Name']
        try:
            driver.get(link)
            time.sleep(2)
            try:
                view_buses_buttons = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']")))
                for button in view_buses_buttons:
                    wait.until(EC.element_to_be_clickable(button)).click()
                    time.sleep(3)  # Wait for the buses to load
            except Exception as e:
                print(f"Error clicking 'View Buses' button for {route_name}: {e}")

            bus_containers = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'clearfix bus-item')]")))
            for bus_container in bus_containers:
                try:
                    busname = bus_container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                    bustype = bus_container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                    departuretiming = bus_container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                    duration = bus_container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                    reachtiming = bus_container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
                    rating = bus_container.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
                    price = bus_container.find_element(By.XPATH, ".//span[contains(@class,'f-19 f-bold') or contains(@class,'f-bold f-19')]").text
                    seat = bus_container.find_element(By.XPATH, ".//div[contains(@class, 'seat-left m-top-30') or contains(@class,'seat-left m-top-16')]").text

                    details = {
                        'State': "Bihar",
                        'Route Name': route_name,
                        'Route Link': link,
                        'Bus Name': busname,
                        'Bus Type': bustype,
                        'Departing Time': departuretiming,
                        'Duration': duration,
                        'Reaching Time': reachtiming,
                        'Star Rating': rating,
                        'Price': price,
                        'Seat Availability': seat,
                    }
                     # Check if the details already exist in the list
                    if details not in alldetails:
                        print(details)
                        alldetails.append(details)

                        # Insert data into the database
                        insert_query = """
                        INSERT IGNORE INTO BusDetails (state, route_name, route_link, bus_name, bus_type, departing_time, duration, reaching_time, star_rating, price, seat_availability)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        """
                        cursor.execute(insert_query, (
                            details['State'],
                            details['Route Name'],
                            details['Route Link'],
                            details['Bus Name'],
                            details['Bus Type'],
                            details['Departing Time'],
                            details['Duration'],
                            details['Reaching Time'],
                            details['Star Rating'],
                            details['Price'],
                            details['Seat Availability']
                        ))
                        db_connection.commit()
                except Exception as e:
                    print(f"Error scraping bus details for {route_name}: {e}")
        except Exception as e:
            print(f"Error accessing route link {link}: {e}")
    return alldetails

# Main execution
routes = navigate_pages()
print("Scraped Routes:", routes)
bus_details = collect_bus_details(routes)
print("Collected Bus Details:", bus_details)

df = pd.DataFrame(routes)
df.to_csv('BIHAR.csv', index=False)
driver.quit()
cursor.close()
db_connection.close()




Pagination error at page 2: Message: element click intercepted: Element is not clickable at point (719, 2094)
  (Session info: chrome=132.0.6834.111)
Stacktrace:
	GetHandleVerifier [0x00007FF65851CC05+28741]
	(No symbol) [0x00007FF65847FED0]
	(No symbol) [0x00007FF658314FBA]
	(No symbol) [0x00007FF658370F88]
	(No symbol) [0x00007FF65836E9D2]
	(No symbol) [0x00007FF65836BAE1]
	(No symbol) [0x00007FF65836AA41]
	(No symbol) [0x00007FF65835C514]
	(No symbol) [0x00007FF65838FE2A]
	(No symbol) [0x00007FF65835BDE6]
	(No symbol) [0x00007FF658390040]
	(No symbol) [0x00007FF6583AFFE1]
	(No symbol) [0x00007FF65838FBD3]
	(No symbol) [0x00007FF658359F43]
	(No symbol) [0x00007FF65835B2D1]
	GetHandleVerifier [0x00007FF65884DE3D+3375741]
	GetHandleVerifier [0x00007FF65886096A+3452330]
	GetHandleVerifier [0x00007FF65885671D+3410781]
	GetHandleVerifier [0x00007FF6585E653B+854395]
	(No symbol) [0x00007FF65848BE0F]
	(No symbol) [0x00007FF658487714]
	(No symbol) [0x00007FF6584878AD]
	(No symbol) [0x00007FF